In [65]:
%matplotlib notebook

import matplotlib.pyplot as plt
from IPython.display import display
import neurokit2 as nk
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.signal
import os
import pickle

# Set matplotlib parameters for displaying graphs
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = [10, 5]  # Bigger images
plt.rcParams['font.size']= 14

pd.set_option('display.max_columns', None)

In [3]:
cwd = os.getcwd()
home_dir = os.path.dirname(cwd)

In [4]:
filepaths = []
for subdir, dirs, files in os.walk(home_dir):
    for filename in files:
        filepath = subdir + os.sep + filename
        
        if filepath.endswith(".acq"):
            filepaths.append(filepath)
filepaths

['C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part201\\part201_baseline.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part201\\part201_pilot.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part202\\part202_pilot.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part206\\part206_pilot.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part207\\part207_pilot.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part208\\part208_pilot.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part209\\part209_pilot.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part210\\part210_pilot.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part211\\part211_pilot.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part212\\part212_pilot.acq',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part213\\part2

In [28]:
#Process data files into ECG,EDA,R

for file in filepaths:
    # Read in data directly from .acq file and rename dataframe columns
    # ACQ file has pre-named colums so you need to make them easier to deal with later in the script
    # subject numbers are 201-215
    # raw data is barely useful, in units of voltage (RSP), microsiemens (EDA), mV (ECG), and V (TRIG)

    data, sampling_rate = nk.read_acqknowledge(file)
    data = data.rename(columns={"RSP, X, RSPEC-R": "RSP", "DTU100 - Trigger View, AMI / HLT - A11": "TRIG",
                                "EDA, X, PPGED-R": "EDA", "ECG, X, RSPEC-R": "ECG"})
    
    # Process ECG, EDA, and RSP over the entire experiment
    # takes the unidimensional psychophys data and derives relevant features for each subject
    ecg_signals, info1 = nk.ecg_process(data["ECG"], sampling_rate=2000)
    eda_signals, info2 = nk.eda_process(data["EDA"], sampling_rate=2000)
    rsp_signals, info3 = nk.rsp_process(data["RSP"], sampling_rate=2000)
    
    pickle.dump( ecg_signals, open( file[:-10]+"_ecg_signals.p", "wb" ) )
    pickle.dump( eda_signals, open( file[:-10]+"_eda_signals.p", "wb" ) )
    pickle.dump( rsp_signals, open( file[:-10]+"_rsp_signals.p", "wb" ) )

OSError: [Errno 28] No space left on device

In [56]:
#Get picklenames
pickle_signals = []
for subdir, dirs, files in os.walk(home_dir):
    for filename in files:
        filepath = subdir + os.sep + filename
        
        if filepath.endswith(".p"):
            pickle_signals.append(filepath)
pickle_signals

['C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part201\\part201_baseline_ecg_signals.p',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part201\\part201_baseline_eda_signals.p',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part201\\part201_baseline_rsp_signals.p',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part201\\part201_ecg_signals.p',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part201\\part201_eda_signals.p',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part201\\part201_rsp_signals.p',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part202\\part202_ecg_signals.p',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part202\\part202_eda_signals.p',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part202\\part202_rsp_signals.p',
 'C:\\Users\\JZ\\Documents\\Graduate School\\HOME NSTRI\\data\\part206\\part206_ecg_signals.p',
 'C:\\Users\\

In [71]:
#Code to verify pickling worked!
ecg_signals = []
eda_signals = []
rsp_signals = []
for p in pickle_signals:
    if "ecg" in p:
        ecg = pickle.load(open(p,"rb"))
        ecg_signals.append(ecg)
    if "eda" in p:
        eda = pickle.load(open(p,"rb"))
        eda_signals.append(eda)
    if "rsp" in p:
        rsp = pickle.load(open(p,"rb"))
        rsp_signals.append(rsp)